In [1]:
import pandas as pd
import glob
import holidays
import datetime
from sklearn.ensemble import RandomForestRegressor
from numpy import mean
#Bokhoven level
path3 = "../data/waterschap-aa-en-maas_sewage_2019/sewer_data/data_pump/RG8180_L0/"
#Bokhoven flow
path4 = "../data/waterschap-aa-en-maas_sewage_2019/sewer_data/data_pump/RG8180_Q0/"
path6 = "../data/waterschap-aa-en-maas_sewage_2019/sewer_data/data_pump/rg8170_99/"

In [3]:
flow_haarsteeg = pd.concat([pd.read_csv(file) for file in glob.glob(path6+"/*.*")], ignore_index = True)

features = flow_haarsteeg[["datumBeginMeting", "datumEindeMeting", "hstWaarde"]].sort_values(by='datumBeginMeting')
features["datumBeginMeting"] = pd.to_datetime(features["datumBeginMeting"])
features["datumEindeMeting"] = pd.to_datetime(features["datumEindeMeting"])
features = features.set_index("datumBeginMeting", drop = False).resample("60Min").sum()
features = features.reset_index()
features["day"] = features["datumBeginMeting"].dt.day
features["hour"] = features["datumBeginMeting"].dt.hour
features["dayofyear"] = features["datumBeginMeting"].dt.dayofyear
features["dayofweek"] =  features["datumBeginMeting"].dt.dayofweek


nl_holidays = holidays.CountryHoliday('NL')

holiday = []
for i in features["datumBeginMeting"]:
    if i in nl_holidays:
        holiday.append(1)
    else:
        holiday.append(0)

features["holiday"] = holiday
            
train1 = features[(features["datumBeginMeting"] >= min(features["datumBeginMeting"])) &
                 (features["datumBeginMeting"] <= pd.Timestamp(2019, 1, 1))]
train = train1[["hour"]]
test1 = features[features["datumBeginMeting"] > pd.Timestamp(2019, 1, 1)]
test = test1[["hour"]]

rf = RandomForestRegressor(n_estimators = 1000)
rf.fit(train, train1["hstWaarde"])
prediction = rf.predict(test)
error = test1["hstWaarde"] - prediction
def mse(d):
    """Mean Squared Error"""
    return mean(d * d)

mse(error)

1034661906.9185008

In [12]:
1037152077.6889356

8761    -20100.389453
8762     -2967.323357
8763    -13799.428245
8764    -11207.535368
8765    -15732.980468
8766     -6424.735561
8767    -18281.235298
8768    -19501.482832
8769    -23235.096889
8770    -28986.131612
8771    -16463.047471
8772      1647.434489
8773      3890.993034
8774     13406.853010
8775      7510.018138
8776      -870.175689
8777     -2385.476111
8778     -5317.444390
8779    -10961.664740
8780    -11862.275814
8781     -8169.243747
8782      -302.765039
8783    -17566.609815
8784     -4337.660823
8785    -20052.189453
8786     -6919.423357
8787    -12922.028245
8788     -6936.235368
8789    -26500.780468
8790    -16900.935561
             ...     
14370    -9795.944390
14371   -18061.264740
14372    -5264.175814
14373    -7490.243747
14374   -13769.465039
14375    -4282.309815
14376   -10359.160823
14377   -15599.289453
14378   -18359.523357
14379   -10924.528245
14380   -16778.935368
14381   -14926.380468
14382   -15389.935561
14383   -27933.735298
14384   -1

In [76]:
test1["predictions"] = prediction

C:\Users\Hermii\Anaconda3\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
